# Random Forest Classification

## Importing the libraries

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

In [2]:
%cd Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\UPSIT_Processed_Files
df = pd.read_csv('UPSIT3Categ_Methylome_APPRDXFilter.csv')
print(df.shape)
df.head()

Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\UPSIT_Processed_Files
(457, 5678)


,Sentrix,PATNO,HYS,Gender,APPRDX,UPSIT_3,cg06067658,cg15442538,cg25928446,cg12415575,...,cg05766881,cg01708543,cg10933281,cg04735449,cg15856662,cg11864127,cg14420798,cg08859349,cg01481205,cg19228755
0,200991620021_R01C01,3001,2,1,1,1,0.088679,0.890148,0.719238,0.066678,...,0.748763,0.830052,0.861102,0.855659,0.788271,0.805084,0.729165,0.803183,0.925666,0.849608
1,200973410167_R04C01,3007,2,1,1,2,0.128668,0.916448,0.726893,0.066768,...,0.886279,0.898054,0.896800,0.911732,0.863749,0.893595,0.829444,0.866504,0.917233,0.786606
2,200973410167_R05C01,3010,2,1,1,2,0.106438,0.922987,0.715904,0.061755,...,0.847166,0.880457,0.909545,0.902309,0.848555,0.881235,0.840847,0.886343,0.912116,0.786117
3,200989060001_R06C01,3012,2,1,1,2,0.162354,0.913700,0.768198,0.048826,...,0.888852,0.905171,0.911937,0.901597,0.896735,0.899922,0.877522,0.884145,0.943558,0.914531
4,200989060143_R08C01,3024,2,1,1,0,0.103971,0.904105,0.803156,0.043982,...,0.848564,0.898804,0.879129,0.873831,0.851764,0.869911,0.857320,0.868054,0.941743,0.892600


In [3]:
df1 = df.drop(['PATNO', 'Sentrix', 'HYS', 'Gender', 'UPSIT_3'], axis=1)
df1.head(2)

,APPRDX,cg06067658,cg15442538,cg25928446,cg12415575,cg26358225,cg19058526,cg20195763,cg23725394,cg27107076,...,cg05766881,cg01708543,cg10933281,cg04735449,cg15856662,cg11864127,cg14420798,cg08859349,cg01481205,cg19228755
0,1,0.088679,0.890148,0.719238,0.066678,0.151532,0.133249,0.479724,0.926745,0.538423,...,0.748763,0.830052,0.861102,0.855659,0.788271,0.805084,0.729165,0.803183,0.925666,0.849608
1,1,0.128668,0.916448,0.726893,0.066768,0.198886,0.141136,0.412058,0.876792,0.665097,...,0.886279,0.898054,0.896800,0.911732,0.863749,0.893595,0.829444,0.866504,0.917233,0.786606


In [4]:
df1 = df1.transpose()
df1.head(2)

,0,1,2,3,4,5,6,7,8,9,...,447,448,449,450,451,452,453,454,455,456
APPRDX,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
cg06067658,0.088679,0.128668,0.106438,0.162354,0.103971,0.102366,0.079617,0.137612,0.078097,0.107534,...,0.139252,0.089804,0.124472,0.117517,0.105056,0.109891,0.093902,0.076561,0.068688,0.077614


In [5]:
df1.rename(columns=df1.iloc[0], inplace = True)


In [6]:
df1.index.name ='CpG'
df1.reset_index(inplace=True)
df1.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
0,APPRDX,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
1,cg06067658,0.088679,0.128668,0.106438,0.162354,0.103971,0.102366,0.079617,0.137612,0.078097,...,0.139252,0.089804,0.124472,0.117517,0.105056,0.109891,0.093902,0.076561,0.068688,0.077614


In [7]:
df1 = df1.drop ([0], axis = 0)
df1.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,cg06067658,0.088679,0.128668,0.106438,0.162354,0.103971,0.102366,0.079617,0.137612,0.078097,...,0.139252,0.089804,0.124472,0.117517,0.105056,0.109891,0.093902,0.076561,0.068688,0.077614
2,cg15442538,0.890148,0.916448,0.922987,0.913700,0.904105,0.929633,0.911634,0.918231,0.890827,...,0.905983,0.934486,0.901157,0.917400,0.938586,0.906540,0.904645,0.919509,0.934254,0.914985


In [8]:
df2 = pd.read_csv ("UPSIT_NMI_Top50.csv")
df2.head(2)

,CpG
0,cg03401738
1,cg20722088


In [9]:
df3 = pd.merge (df1, df2, on= "CpG", how= "inner")
df3.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
0,cg08744312,0.902975,0.904315,0.901686,0.895425,0.906603,0.920725,0.906283,0.914591,0.910144,...,0.914695,0.927297,0.928154,0.926816,0.926646,0.892343,0.914455,0.900380,0.931231,0.904778
1,cg22295787,0.016057,0.021968,0.021522,0.011719,0.008112,0.011084,0.010186,0.011982,0.021868,...,0.014281,0.012876,0.015690,0.017897,0.007996,0.010125,0.017560,0.016737,0.014842,0.014695


In [10]:
df3 = df3.transpose()
df3.head(2)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
CpG,cg08744312,cg22295787,cg19997384,cg02295000,cg19400616,cg27344018,cg12038759,cg01232534,cg20118247,cg03682895,...,cg05516535,cg11485247,cg04581024,cg03191458,cg20003494,cg05142211,cg19731340,cg06794504,cg13959611,cg14149699
1.0,0.902975,0.016057,0.284672,0.826318,0.909653,0.845686,0.739036,0.275509,0.937667,0.902725,...,0.930609,0.024186,0.884052,0.921355,0.025708,0.035018,0.018411,0.752823,0.044262,0.030215


In [11]:
df3.rename(columns=df3.iloc[0], inplace = True)


In [12]:
df3.index.name ='APPRDX'
df3.reset_index(inplace=True)
df3.head(2)

,APPRDX,cg08744312,cg22295787,cg19997384,cg02295000,cg19400616,cg27344018,cg12038759,cg01232534,cg20118247,...,cg05516535,cg11485247,cg04581024,cg03191458,cg20003494,cg05142211,cg19731340,cg06794504,cg13959611,cg14149699
0,CpG,cg08744312,cg22295787,cg19997384,cg02295000,cg19400616,cg27344018,cg12038759,cg01232534,cg20118247,...,cg05516535,cg11485247,cg04581024,cg03191458,cg20003494,cg05142211,cg19731340,cg06794504,cg13959611,cg14149699
1,1.0,0.902975,0.016057,0.284672,0.826318,0.909653,0.845686,0.739036,0.275509,0.937667,...,0.930609,0.024186,0.884052,0.921355,0.025708,0.035018,0.018411,0.752823,0.044262,0.030215


In [13]:
df3 = df3.drop ([0], axis = 0)
df3.head(2)

,APPRDX,cg08744312,cg22295787,cg19997384,cg02295000,cg19400616,cg27344018,cg12038759,cg01232534,cg20118247,...,cg05516535,cg11485247,cg04581024,cg03191458,cg20003494,cg05142211,cg19731340,cg06794504,cg13959611,cg14149699
1,1.0,0.902975,0.016057,0.284672,0.826318,0.909653,0.845686,0.739036,0.275509,0.937667,...,0.930609,0.024186,0.884052,0.921355,0.025708,0.035018,0.018411,0.752823,0.044262,0.030215
2,1.0,0.904315,0.021968,0.32901,0.805755,0.942176,0.810976,0.578553,0.225155,0.927726,...,0.925093,0.031999,0.915653,0.911271,0.045513,0.041906,0.017375,0.843436,0.033433,0.083898


In [14]:
df3.reset_index(inplace = True, drop = True)
df3.head()

,APPRDX,cg08744312,cg22295787,cg19997384,cg02295000,cg19400616,cg27344018,cg12038759,cg01232534,cg20118247,...,cg05516535,cg11485247,cg04581024,cg03191458,cg20003494,cg05142211,cg19731340,cg06794504,cg13959611,cg14149699
0,1.0,0.902975,0.016057,0.284672,0.826318,0.909653,0.845686,0.739036,0.275509,0.937667,...,0.930609,0.024186,0.884052,0.921355,0.025708,0.035018,0.018411,0.752823,0.044262,0.030215
1,1.0,0.904315,0.021968,0.32901,0.805755,0.942176,0.810976,0.578553,0.225155,0.927726,...,0.925093,0.031999,0.915653,0.911271,0.045513,0.041906,0.017375,0.843436,0.033433,0.083898
2,1.0,0.901686,0.021522,0.334271,0.79669,0.944822,0.839075,0.647153,0.225984,0.934948,...,0.915504,0.030786,0.923505,0.917228,0.044374,0.019257,0.015523,0.836201,0.046769,0.042991
3,1.0,0.895425,0.011719,0.345518,0.827471,0.954486,0.892227,0.905273,0.205654,0.940935,...,0.922088,0.026719,0.908377,0.931585,0.033958,0.018673,0.011753,0.887629,0.051056,0.034968
4,1.0,0.906603,0.008112,0.441172,0.846521,0.958551,0.88312,0.744554,0.226375,0.941965,...,0.927286,0.016199,0.922944,0.930641,0.024791,0.024418,0.010823,0.841288,0.035087,0.033764


In [15]:
df3.shape

(457, 51)

In [16]:
df3 ['APPRDX'] = df3 ['APPRDX'].apply(int)


## RanFor 1000 times, CpG only 

In [17]:
df3.head(2)

,APPRDX,cg08744312,cg22295787,cg19997384,cg02295000,cg19400616,cg27344018,cg12038759,cg01232534,cg20118247,...,cg05516535,cg11485247,cg04581024,cg03191458,cg20003494,cg05142211,cg19731340,cg06794504,cg13959611,cg14149699
0,1,0.902975,0.016057,0.284672,0.826318,0.909653,0.845686,0.739036,0.275509,0.937667,...,0.930609,0.024186,0.884052,0.921355,0.025708,0.035018,0.018411,0.752823,0.044262,0.030215
1,1,0.904315,0.021968,0.32901,0.805755,0.942176,0.810976,0.578553,0.225155,0.927726,...,0.925093,0.031999,0.915653,0.911271,0.045513,0.041906,0.017375,0.843436,0.033433,0.083898


In [18]:
df3.shape

(457, 51)

In [19]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [20]:
dframe1.head(2)

,Accuracy,Std_Dev
0,0.677,0.109640
1,0.635,0.103078


In [21]:
dframe1.to_csv ("UPSIT_NMITop50CpG_Accuracy_1000RF.csv")